# Tool 1 - Data Ingest Demo (LangGraph Nodes)

**Purpose:** Map business entities from Tool 0 to BS metadata candidates using LangGraph agent with nodes.

**LangGraph Features Tested:**
- ✅ Agent with discrete nodes (LangGraph way of thinking)
- ✅ **Multiple LLM nodes** (prepare = ranking, mapping = matching)
- ✅ Shared state between nodes
- ✅ Structured output (ToolStrategy) in multiple contexts
- ✅ Dynamic prompt middleware (inject scope_out)
- ✅ Streaming progress between nodes

**Architecture:**
```
Load Node → Prepare Node (LLM) → Mapping Node (LLM) → Filter Node → Save Node
     ↓            ↓                     ↓                  ↓           ↓
 Tool 0 JSON  LLM Ranking         LLM Matching        Blacklist    Results
 + BS export  (relevance)       (confidence+rationale) (deterministic) (JSON)
```

**Model:** openai:gpt-5-mini (consistent across all nodes)

**Showcase:** Two intelligent LLM nodes with different purposes - demonstrates "each node does one thing well"

In [1]:
# Install required packages (run once)
# !pip install langgraph langchain langchain-openai pydantic python-dotenv

In [2]:
# Import required modules
from pydantic import BaseModel, Field
from datetime import datetime
from pathlib import Path
from typing import Annotated
import json

from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import dynamic_prompt
from langchain.agents.structured_output import ToolStrategy
from langgraph.graph import StateGraph, START, END

print("✅ Imports successful")

✅ Imports successful


## 1. Define Schemas & State

Pydantic schemas for structured output + LangGraph state.

In [3]:
# Pydantic schemas for structured output
class EntityMapping(BaseModel):
    """Single entity mapped to a BS candidate."""

    entity: str = Field(description="Original entity name from Tool 0")
    candidate_id: str = Field(description="BS candidate identifier (e.g., 'dm_bs_suppliers')")
    candidate_name: str = Field(description="Human-readable candidate name")
    confidence: float = Field(
        description="Confidence score 0.0-1.0",
        ge=0.0,
        le=1.0
    )
    rationale: str = Field(description="Why this mapping was suggested")


class MappingSuggestions(BaseModel):
    """Complete set of entity mappings."""

    mappings: list[EntityMapping] = Field(
        description="List of entity-to-candidate mappings"
    )


# LangGraph state (shared across all nodes)
class Tool1State(AgentState):
    """Shared state for Tool 1 graph."""

    # Input data
    entities: list[str]
    scope_out: str
    candidates: list[dict]

    # Processing results
    raw_mappings: list[dict] | None = None
    filtered_mappings: list[dict] | None = None

    # Output paths
    output_json_path: str | None = None
    output_artifact_path: str | None = None

print("✅ Schemas & state defined")

✅ Schemas & state defined


## 2. Node 1: Load Data

Load Tool 0 JSON (entities, scope_out) + BS export JSON (candidates).

In [4]:
def load_node(state: Tool1State) -> dict:
    """Load Tool 0 output and BS export."""
    print("📂 Node 1: Loading data...")

    # Load Tool 0 JSON
    tool0_path = Path.cwd().parent / 'data' / 'tool0_samples' / '2025-10-31T01:14:27.960789.json'
    with open(tool0_path, 'r', encoding='utf-8') as f:
        tool0_data = json.load(f)

    entities = tool0_data.get('entities', [])
    scope_out = tool0_data.get('scope_out', '')

    print(f"   ✅ Loaded {len(entities)} entities from Tool 0")
    print(f"   ✅ Scope out: {scope_out[:100]}...")

    # Load BS export JSON (pre-filtered externally: flattened, deduped, BS-only)
    bs_path = Path.cwd().parent / 'docs_langgraph' / 'BA-BS_Datamarts_metadata.json'
    with open(bs_path, 'r', encoding='utf-8') as f:
        bs_data = json.load(f)

    # Flatten array-of-arrays structure
    if isinstance(bs_data, list) and len(bs_data) > 0 and isinstance(bs_data[0], list):
        candidates_flat = bs_data[0]  # Extract inner array
    else:
        candidates_flat = bs_data

    # Filter to BS-only (dm_bs_* prefix) - FIX: use 'displayName' not 'name'
    candidates = [
        c for c in candidates_flat
        if isinstance(c, dict) and c.get('displayName', '').startswith('dm_bs_')
    ]

    print(f"   ✅ Loaded {len(candidates)} BS candidates")
    if candidates:
        print(f"   📋 Candidates:")
        for c in candidates:
            print(f"      - {c.get('displayName')}: {c.get('description', 'N/A')[:60]}...")

    return {
        'entities': entities,
        'scope_out': scope_out,
        'candidates': candidates
    }

print("✅ Load node defined")

✅ Load node defined


## 3. Node 2: Prepare Candidates (LLM Ranking)

**LLM-based prefiltering:** Rank candidates by relevance for entities (showcase multiple LLM nodes).

In [5]:
# Pydantic schema for candidate ranking
class CandidateRank(BaseModel):
    """Single candidate with relevance score."""

    candidate_id: str = Field(description="Candidate identifier")
    relevance_score: float = Field(
        description="Relevance score 0.0-1.0",
        ge=0.0,
        le=1.0
    )
    reason: str = Field(description="Why this candidate is relevant")


class CandidateRanking(BaseModel):
    """Ranked list of candidates."""

    ranked_candidates: list[CandidateRank] = Field(
        description="Candidates ranked by relevance"
    )


def prepare_node(state: Tool1State) -> dict:
    """LLM ranks candidates by relevance for entities."""
    print("🔧 Node 2: LLM ranking candidates...")

    entities = state['entities']
    candidates = state['candidates']

    print(f"   Input: {len(entities)} entities, {len(candidates)} candidates")

    # Prepare candidate summaries for LLM
    candidate_summaries = []
    for c in candidates:
        desc = c.get('description', '')
        candidate_summaries.append({
            'id': c.get('displayName', 'unknown'),
            'name': c.get('fullName', c.get('displayName', 'unknown')),
            'description': desc[:200] + '...' if len(desc) > 200 else desc
        })

    # Create lightweight ranking agent
    ranking_agent = create_agent(
        model="openai:gpt-5-mini",
        response_format=ToolStrategy(CandidateRanking),
        tools=[],
        system_prompt="""You are a data relevance analyzer.
Rank database candidates by relevance for the given business entities.

Consider:
- Semantic similarity between entity names and candidate names/descriptions
- Domain relevance (e.g., "suppliers" → purchasing schemas)
- Czech/English terminology overlap

Return relevance scores (0.0-1.0) and reasons."""
    )

    # Prepare ranking request
    ranking_request = f"""Rank these candidates by relevance for the business entities:

**Business Entities:**
{json.dumps(entities, indent=2, ensure_ascii=False)}

**Available Candidates:**
{json.dumps(candidate_summaries, indent=2, ensure_ascii=False)}

Return ranked candidates with relevance scores."""

    # Invoke ranking agent
    result = ranking_agent.invoke({
        "messages": [
            {"role": "user", "content": ranking_request}
        ]
    })

    # Extract structured response
    structured_response = result.get('structured_response')
    if not structured_response:
        raise ValueError("No structured response from ranking agent")

    # Convert to dict
    ranking_data = (
        structured_response.model_dump()
        if hasattr(structured_response, 'model_dump')
        else structured_response.dict()
    )

    ranked_list = ranking_data.get('ranked_candidates', [])

    # Take top N (or all if fewer) - for demo, take all with score > 0.3
    relevant_candidates = [
        r for r in ranked_list
        if r['relevance_score'] > 0.3
    ]

    # Match back to full candidate objects
    ranked_candidate_ids = [r['candidate_id'] for r in relevant_candidates]
    prepared_candidates = [
        {
            'id': c.get('displayName', 'unknown'),
            'name': c.get('fullName', c.get('displayName', 'unknown')),
            'description': c.get('description', '')[:200] + '...'
                if len(c.get('description', '')) > 200
                else c.get('description', ''),
            'relevance_score': next(
                (r['relevance_score'] for r in relevant_candidates if r['candidate_id'] == c.get('displayName')),
                0.0
            )
        }
        for c in candidates
        if c.get('displayName') in ranked_candidate_ids
    ]

    print(f"   ✅ Ranked {len(ranked_list)} candidates")
    print(f"   ✅ Selected {len(prepared_candidates)} relevant (score > 0.3)")

    return {
        'candidates': prepared_candidates
    }

print("✅ Prepare node (LLM ranking) defined")

✅ Prepare node (LLM ranking) defined


## 4. Node 3: LLM Mapping (Agent with Middleware)

Use LangGraph agent with:
- Structured output (ToolStrategy)
- Dynamic prompt middleware (inject scope_out)

In [6]:
def mapping_node(state: Tool1State) -> dict:
    """Map entities to candidates using LLM agent."""
    print("🤖 Node 3: LLM mapping with agent...")

    entities = state['entities']
    candidates = state['candidates']
    scope_out = state['scope_out']

    # Dynamic prompt middleware (inject scope_out)
    @dynamic_prompt
    def scope_aware_prompt(request) -> str:
        return f"""You are an entity mapper. Map business entities to database candidates using fuzzy matching.

Consider:
- Synonyms (e.g., "dodavatelé" = "suppliers")
- Czech/English variants
- Partial name matches
- Description context

IMPORTANT: Avoid candidates related to these excluded topics:
{scope_out}

Return confidence scores (0.0-1.0) and rationale for each mapping."""

    # Create agent with structured output
    agent = create_agent(
        model="openai:gpt-5-mini",
        response_format=ToolStrategy(MappingSuggestions),
        tools=[],  # No tools needed for single-shot mapping
        middleware=[scope_aware_prompt]
    )

    # Prepare user message
    user_message = f"""Map these entities to the best matching candidates:

**Entities to map:**
{json.dumps(entities, indent=2, ensure_ascii=False)}

**Available candidates:**
{json.dumps(candidates, indent=2, ensure_ascii=False)}

Return mappings with confidence scores and rationale."""

    # Invoke agent
    result = agent.invoke({
        "messages": [
            {"role": "user", "content": user_message}
        ]
    })

    # Extract structured response
    structured_response = result.get('structured_response')
    if not structured_response:
        raise ValueError("No structured response from agent")

    # Convert to dict
    mappings_data = (
        structured_response.model_dump()
        if hasattr(structured_response, 'model_dump')
        else structured_response.dict()
    )

    raw_mappings = mappings_data.get('mappings', [])

    print(f"   ✅ Generated {len(raw_mappings)} mappings")

    return {
        'raw_mappings': raw_mappings
    }

print("✅ Mapping node defined")

✅ Mapping node defined


## 5. Node 4: Filter Blacklist

Apply deterministic scope_out blacklist filter (historical, logs, security terms).

In [7]:
def filter_node(state: Tool1State) -> dict:
    """Filter mappings using scope_out blacklist."""
    print("🔍 Node 4: Applying blacklist filter...")

    raw_mappings = state['raw_mappings']
    scope_out = state['scope_out'].lower()

    # Define blacklist keywords
    blacklist = ['historical', 'logs', 'security', 'archive', 'audit']

    # Also extract keywords from scope_out
    scope_keywords = [
        word.strip().lower()
        for word in scope_out.split()
        if len(word.strip()) > 3
    ]

    all_blacklist = set(blacklist + scope_keywords)

    # Filter mappings
    filtered = []
    for mapping in raw_mappings:
        candidate_name = mapping.get('candidate_name', '').lower()
        candidate_id = mapping.get('candidate_id', '').lower()

        # Check if any blacklist term appears
        is_blacklisted = any(
            term in candidate_name or term in candidate_id
            for term in all_blacklist
        )

        if not is_blacklisted:
            filtered.append(mapping)

    removed_count = len(raw_mappings) - len(filtered)
    print(f"   ✅ Filtered: {len(filtered)} kept, {removed_count} removed")

    return {
        'filtered_mappings': filtered
    }

print("✅ Filter node defined")

✅ Filter node defined


## 6. Node 5: Save Results

Save filtered_dataset.json + ingest summary to artifacts.

In [8]:
def save_node(state: Tool1State) -> dict:
    """Save filtered dataset and artifacts."""
    print("💾 Node 5: Saving results...")

    entities = state['entities']
    filtered_mappings = state['filtered_mappings']
    scope_out = state['scope_out']

    timestamp = datetime.now().isoformat()

    # Prepare filtered dataset
    filtered_dataset = {
        'timestamp': timestamp,
        'business_context': {
            'entities': entities,
            'scope_out': scope_out
        },
        'mappings': filtered_mappings,
        'stats': {
            'total_entities': len(entities),
            'mapped_entities': len(filtered_mappings),
            'avg_confidence': (
                sum(m['confidence'] for m in filtered_mappings) / len(filtered_mappings)
                if filtered_mappings else 0.0
            )
        }
    }

    # Save to data/tool1/
    output_dir = Path.cwd().parent / 'data' / 'tool1'
    output_dir.mkdir(parents=True, exist_ok=True)

    json_path = output_dir / 'filtered_dataset.json'
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(filtered_dataset, f, indent=2, ensure_ascii=False)

    print(f"   ✅ Dataset saved: {json_path}")

    # Save artifacts summary
    artifacts_dir = Path.cwd().parent / 'scrum' / 'artifacts'
    artifacts_dir.mkdir(parents=True, exist_ok=True)

    artifact_path = artifacts_dir / f"{timestamp.split('T')[0]}_tool1-ingest-summary.json"
    with open(artifact_path, 'w', encoding='utf-8') as f:
        json.dump(filtered_dataset['stats'], f, indent=2)

    print(f"   ✅ Artifact saved: {artifact_path}")

    return {
        'output_json_path': str(json_path),
        'output_artifact_path': str(artifact_path)
    }

print("✅ Save node defined")

✅ Save node defined


## 7. Build LangGraph

Connect nodes: Load → Prepare → Mapping → Filter → Save

In [9]:
# Build state graph
builder = StateGraph(Tool1State)

# Add nodes
builder.add_node('load', load_node)
builder.add_node('prepare', prepare_node)
builder.add_node('mapping', mapping_node)
builder.add_node('filter', filter_node)
builder.add_node('save', save_node)

# Define edges
builder.add_edge(START, 'load')
builder.add_edge('load', 'prepare')
builder.add_edge('prepare', 'mapping')
builder.add_edge('mapping', 'filter')
builder.add_edge('filter', 'save')
builder.add_edge('save', END)

# Compile graph
graph = builder.compile()

print("✅ LangGraph compiled")
print("\n📊 Graph structure:")
print("   START → load → prepare → mapping → filter → save → END")

✅ LangGraph compiled

📊 Graph structure:
   START → load → prepare → mapping → filter → save → END


## 8. Execute Graph

Run the complete pipeline and stream progress.

In [10]:
print("🚀 Executing Tool 1 pipeline...\n")
print("=" * 60)

# Execute graph
final_state = graph.invoke({})

print("=" * 60)
print("\n✅ Pipeline complete!")
print(f"\n📊 Final Results:")
print(f"   Entities processed: {len(final_state.get('entities', []))}")
print(f"   Mappings generated: {len(final_state.get('raw_mappings', []))}")
print(f"   Mappings after filter: {len(final_state.get('filtered_mappings', []))}")
print(f"   Output JSON: {final_state.get('output_json_path')}")
print(f"   Artifact: {final_state.get('output_artifact_path')}")

🚀 Executing Tool 1 pipeline...

📂 Node 1: Loading data...
   ✅ Loaded 4 entities from Tool 0
   ✅ Scope out: HR data o zaměstnancích; Finanční forecasting a budgetování; Real-time monitoring dodávek; Integrace...
   ✅ Loaded 1 BS candidates
   📋 Candidates:
      - dm_bs_purchase: BS (Production Purchasing) Reporting Data Mart...
🔧 Node 2: LLM ranking candidates...
   Input: 4 entities, 1 candidates
   ✅ Ranked 1 candidates
   ✅ Selected 1 relevant (score > 0.3)
🤖 Node 3: LLM mapping with agent...
   ✅ Generated 4 mappings
🔍 Node 4: Applying blacklist filter...
   ✅ Filtered: 4 kept, 0 removed
💾 Node 5: Saving results...
   ✅ Dataset saved: /Users/marekminarovic/archi-agent/data/tool1/filtered_dataset.json
   ✅ Artifact saved: /Users/marekminarovic/archi-agent/scrum/artifacts/2025-10-31_tool1-ingest-summary.json

✅ Pipeline complete!

📊 Final Results:
   Entities processed: 4
   Mappings generated: 4
   Mappings after filter: 4
   Output JSON: /Users/marekminarovic/archi-agent/data/too

## 9. Display Results

Show filtered mappings with confidence scores.

In [11]:
# Display filtered mappings
filtered_mappings = final_state.get('filtered_mappings', [])

print("📋 Filtered Entity Mappings:\n")
print("=" * 80)

for i, mapping in enumerate(filtered_mappings, 1):
    print(f"\n{i}. {mapping['entity']} → {mapping['candidate_name']}")
    print(f"   ID: {mapping['candidate_id']}")
    print(f"   Confidence: {mapping['confidence']:.2f}")
    print(f"   Rationale: {mapping['rationale']}")

print("\n" + "=" * 80)

# Calculate stats
if filtered_mappings:
    avg_confidence = sum(m['confidence'] for m in filtered_mappings) / len(filtered_mappings)
    high_confidence = sum(1 for m in filtered_mappings if m['confidence'] >= 0.8)

    print(f"\n📈 Statistics:")
    print(f"   Average confidence: {avg_confidence:.2f}")
    print(f"   High confidence (≥0.8): {high_confidence}/{len(filtered_mappings)}")

📋 Filtered Entity Mappings:


1. Suppliers (dodavatelé) → Systems>dap_gold_prod>dm_bs_purchase
   ID: dm_bs_purchase
   Confidence: 0.92
   Rationale: Strong fuzzy match: 'Suppliers' (Czech: 'dodavatelé') is a core master dimension used by purchasing data marts. The candidate is a Purchasing Reporting Data Mart ('BS (Production Purchasing) Reporting Data Mart'), so it likely contains vendor/supplier master and supplier-related transaction attributes. No conflict with excluded topics.

2. Purchase Orders (nákupní objednávky) → Systems>dap_gold_prod>dm_bs_purchase
   ID: dm_bs_purchase
   Confidence: 0.98
   Rationale: Very high confidence: the candidate is explicitly a 'Purchasing' reporting data mart, so purchase orders (nákupní objednávky) are a primary object in this domain. The name and description match directly. Excluded topics do not apply.

3. Products (produkty) → Systems>dap_gold_prod>dm_bs_purchase
   ID: dm_bs_purchase
   Confidence: 0.80
   Rationale: Moderate confidence: p

## 10. Summary

✅ **LangGraph Features Demonstrated:**
- [x] Agent with discrete nodes (load, prepare, mapping, filter, save)
- [x] **Multiple LLM nodes:**
  - **Prepare Node:** LLM ranking (relevance scores)
  - **Mapping Node:** LLM matching (confidence + rationale)
- [x] Shared state (Tool1State) across all nodes
- [x] Structured output via ToolStrategy in 2 contexts (CandidateRanking, MappingSuggestions)
- [x] Dynamic prompt middleware (inject scope_out blacklist)
- [x] Streaming progress between nodes
- [x] "Each node does one thing well" principle

**Model:** openai:gpt-5-mini (consistent, no dynamic routing)

**2-Stage LLM Approach:**
1. **Prepare Node:** Ranks candidates by relevance (lightweight prefilter)
2. **Mapping Node:** Detailed entity-to-candidate matching with confidence

**Outputs:**
- `data/tool1/filtered_dataset.json` - Complete dataset with mappings
- `scrum/artifacts/YYYY-MM-DD_tool1-ingest-summary.json` - Statistics summary

**Why This Architecture:**
- Showcases LangGraph's multi-node intelligence pattern
- Testuje různé structured output schemas (ranking vs mapping)
- Demonstrates separation of concerns (relevance vs matching)
- Not necessary for 2 candidates, but excellent LangGraph demonstration

**Next Steps:**
1. Run compliance checker: `python3 .claude/skills/langchain/compliance-checker/check.py --file notebooks/tool1_ingest_demo.ipynb`
2. Update story: `scrum/backlog/tool1-data-ingest.md` (`skill_created: true`)
3. Test with different Tool 0 outputs
4. Evaluate ranking quality vs direct mapping